In [1]:
import torch
from torch import nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR
import torchvision
from torchvision import transforms, utils, datasets

import os
import sys
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import h5py
import PIL
import numpy as np
import random
import logging
import gdown
from zipfile import ZipFile
import matplotlib.pyplot as plt

%matplotlib inline

## Parameters

In [2]:
RATIO_VALIDATION_SPLIT = 0.1
BATCH_SIZE = 8
NUM_WROKERS = 4
INPUT_IMG_SIZE = (101, 101)
EPOCH = 50               # train the training data n times, to save time, we just train 1 epoch
LR = 0.001               # learning rate
DATASET_ROOT = './3-class_behavior-reflex'
MODELS_ROOT = './models'
CLASSES = np.loadtxt('class_id.txt', str, delimiter='\n')

## Download dataset

In [3]:
train_dataset_url = 'https://drive.google.com/uc?id=10wJUsO2Eo6aOK-ay-UlBBlf1_Ba1LdTY'
train_dataset_name = '3-class_behavior-reflex'
if not os.path.isdir(train_dataset_name):
    gdown.download(train_dataset_url, output=train_dataset_name + '.zip', quiet=False)
    zip1 = ZipFile(train_dataset_name + '.zip')
    zip1.extractall(train_dataset_name)
    zip1.close()

print 'Finished downloading training dataset.'

Finished downloading training dataset.


## Useful functions: 
dataset visualization and image loading function

In [5]:
def vis_img(batch_data):   
    # show images
    imgs = torchvision.utils.make_grid(batch_data)
    imgs = imgs / 2 + 0.5     # unnormalize
    npimgs = imgs.numpy()
    plt.rcParams['figure.figsize'] = [12, 5]
    plt.imshow(np.transpose(npimgs, (1, 2, 0)))

In [6]:
def load_images_from_folder(folder, classes):
    img_paths = []
    labels = []
    for class_id, class_name in enumerate(classes):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            filename.lower().endswith(('.png', '.jpg', '.jpeg'))
            img_paths.append(os.path.join(class_folder, filename))
            labels.append(class_id)
    return img_paths, labels

In [1]:
def export_dataset_split_txt(train_dataset, val_dataset):
    train_list_filename = 'train_dataset.txt'
    test_list_filename = 'validation_dataset.txt'
    
    if not os.path.isfile(train_list_filename):
        with open(train_list_filename, 'w+') as f:
            for idx, (img, label, p) in enumerate(train_dataset):
                f.write(p + ' ' + str(label.numpy()) + '\n')
    else: print(train_list_filename + 'is existed.')
    
    if not os.path.isfile(test_list_filename):
        with open(test_list_filename, 'w+') as f:
            for idx, (img, label, p) in enumerate(val_dataset):
                f.write(p + ' ' + str(label.numpy()) + '\n')
    else: print(test_list_filename + 'is existed.')

## Custom pytorch dataset class for Trailnet (load dataset from folder)

In [7]:
class TrailnetDataset(Dataset):
    def __init__(self, dataset_root):
        classes = CLASSES
        dataset_folders = self.collect_folders_from_dataset(dataset_root, classes)
        
        self.img_list = []
        self.label_list = []
        cnt = 0
        for p in dataset_folders:
            img_paths, labels = load_images_from_folder(p, classes)
            self.img_list.extend(img_paths)
            self.label_list.extend(labels)
            cnt += len(img_paths)
        print '********** Dataset Info start **********\n'
        print 'Dataset folder: ', dataset_folders
        print 'Output classes: ', classes
        print 'Amount of images: ', cnt
        print '\n*********** Dataset Info end ***********\n'
        
        self.data_transform = transforms.Compose([ 
                                transforms.Resize(INPUT_IMG_SIZE), \
                                transforms.ToTensor(), \
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], \
                                                     std=[1, 1, 1]), \
                                ])
        
    def __len__(self):
        return len(self.img_list)
    
    def collect_folders_from_dataset(self, dataset_root, classes):
        # Implement by BFS
        search_list = [dataset_root, ]
        dataset_folders = [] 
        while len(search_list) != 0:
            root = search_list.pop(0)
            if set(os.listdir(root)) == set(classes):
                dataset_folders.append(root)
            else:
                for folder in os.listdir(root):
                    path = os.path.join(root, folder)
                    if os.path.isdir(path):
                        search_list.append(path)
        return dataset_folders

    def __getitem__(self, index):
        'Generates one sample of data'
        # print self.img_list[index]
        # Select sample, then load data and get label
        path = self.img_list[index]
        img_raw = self.default_loader(path)
        x = self.data_transform(img_raw)
        y = torch.tensor(self.label_list[index])
        return x, y, path
    
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            with PIL.Image.open(f) as img:
                return img.convert('RGB')

    def accimage_loader(self, path):
        try:
            return accimage.Image(path)
        except IOError:
            # Potentially a decoding problem, fall back to PIL.Image
            return pil_loader(path)

    def default_loader(self, path):
        if torchvision.get_image_backend() == 'accimage':
            return self.accimage_loader(path)
        else:
            return self.pil_loader(path)

## Dataset splitter 

[Reference link](https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets
)

If you were confusing what are the differences between the dataset and dataloader in Pytorch, check out the [link](https://discuss.pytorch.org/t/discussion-about-datasets-and-dataloaders/296/6).

In [8]:
full_dataset = TrailnetDataset(dataset_root=DATASET_ROOT)
train_size = int((1 - RATIO_VALIDATION_SPLIT) * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

print 'Size of training dataset:   ', len(train_dataset)
print 'Size of validation dataset: ', len(val_dataset)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=False)

********** Dataset Info start **********

Dataset folder:  ['./3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_red', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_B51_flip', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_gray_flip', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_gray', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/wood_TKU_flip', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_B51', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/brick_red_flip', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/virtual_environment/wood_TKU', './3-class_behavior-reflex/3-class_behavior-reflex_taining_data/real-world_environment/3/YB3_C', './3-class_behavior-reflex/3-class_behavior-reflex_tain

In [10]:
# It takes about 3 mins to export txt file
export_dataset_split_txt(train_dataset, val_dataset)
print 'Finished export training and testing datset list.'

Finished export training and testing datset list.
